<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Generate_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import shutil
import zipfile
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir las rutas de los archivos y directorios
source_file = '/content/drive/MyDrive/TFM/Polipos Segmentados.zip'
destination_dir = '/content/Dataset/Polipos'

# Crear el directorio si no existe
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copiar el archivo
shutil.copy(source_file, destination_dir)
print(f'Archivo copiado a: {os.path.join(destination_dir, os.path.basename(source_file))}')

# Descomprimir el archivo ZIP
zip_path = os.path.join(destination_dir, os.path.basename(source_file))
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_dir)
    print(f'Archivo descomprimido en: {destination_dir}')

Mounted at /content/drive
Archivo copiado a: /content/Dataset/Polipos/Polipos Segmentados.zip
Archivo descomprimido en: /content/Dataset/Polipos


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Definir las rutas de los archivos y directorios
source_dir = '/content/Dataset/Polipos/segmentación polipos 265'
destination_base_dir = '/content/Dataset/Polipos/TrainValTest'

# Crear directorios de destino si no existen
train_dir = os.path.join(destination_base_dir, 'train')
val_dir = os.path.join(destination_base_dir, 'val')
test_dir = os.path.join(destination_base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Listar todas las imágenes en el directorio fuente, excluyendo las que terminan en (1).jpg
all_images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and not f.endswith('(1).jpg')]

# Dividir las imágenes en train (60%), val (20%), y test (20%)
train_images, test_images = train_test_split(all_images, test_size=0.4, random_state=42)
val_images, test_images = train_test_split(test_images, test_size=0.5, random_state=42)

# Función para copiar imágenes a los directorios correspondientes
def copy_images(image_list, destination_dir):
    for image in image_list:
        source_file = os.path.join(source_dir, image)
        destination_file = os.path.join(destination_dir, image)
        shutil.copyfile(source_file, destination_file)

copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

print("División de imágenes completada.")


División de imágenes completada.


In [4]:
import cv2
import numpy as np

def detect_and_draw_red_vertices(image_path, output_path, distance_threshold=10):
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Convertir la imagen al espacio de color HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir el rango de color amarillo en HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Crear una máscara para detectar el color amarillo
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Obtener las posiciones de los píxeles amarillos
    yellow_pixels = np.argwhere(mask_yellow == 255)

    # Definir el rango de color rojo en HSV
    lower_red = np.array([0, 240, 2])
    upper_red = np.array([10, 255, 255])

    # Crear una máscara para detectar el color rojo
    mask_red = cv2.inRange(hsv, lower_red, upper_red)

    # Obtener las posiciones de los píxeles rojos
    red_pixels = np.argwhere(mask_red == 255)

    # Crear una imagen en blanco (255) con las mismas dimensiones que la imagen original
    result = np.ones_like(image) * 255

    # Iterar sobre los píxeles rojos y pintar en negro si están cerca de los píxeles amarillos
    for red_pixel in red_pixels:
        for yellow_pixel in yellow_pixels:
            distance = np.linalg.norm(red_pixel - yellow_pixel)
            if distance <= distance_threshold:
                result[red_pixel[0], red_pixel[1]] = [0, 0, 0]
                break  # Salir del bucle interno si el píxel rojo está cerca de algún píxel amarillo

    # Guardar la imagen resultante
    cv2.imwrite(output_path, result)

# Ruta de la imagen de prueba
image_path = '/content/Dataset/Polipos/TrainValTest/test/100H0013.jpg'

# Ruta de salida para la imagen resultante
output_path = '/content/prueba_Aristas_V2_13.jpg'

# Distancia máxima para considerar que un píxel rojo está cerca de un píxel amarillo
distance_threshold = 10

# Detectar y dibujar los puntos rojos como vértices en negro y guardar la imagen
detect_and_draw_red_vertices(image_path, output_path, distance_threshold)

print(f"Imagen guardada en: {output_path}")


Imagen guardada en: /content/prueba_Aristas_V2_13.jpg
